In [1]:
import pandas as pd
df = pd.read_excel('ADNI combined.xlsx', sheet_name='sample')
#print(df)
sid = df['RID']
grp = df['Group at scan date (1=CN, 2=EMCI, 3=LMCI, 4=AD, 5=SMC)']
age = df['Age at scan']
sex = df['Sex (1=female)']
tiv = df['TIV']
field = df['MRI_Field_Strength']
grpbin = (grp > 1) # 1=CN, ...
amy_bin = df['SUMMARYSUVR_WHOLECEREBNORM_1.11CUTOFF']

In [2]:
import glob
dataAD = sorted(glob.glob('mwp1_MNI/AD/*.nii.gz'))
dataLMCI = sorted(glob.glob('mwp1_MNI/LMCI/*.nii.gz'))
dataCN = sorted(glob.glob('mwp1_MNI/CN/*.nii.gz'))
#dataADNI3 = sorted(glob.glob('mwp1_MNI/ADNI3/*.nii.gz'))
dataFiles = dataAD + dataLMCI + dataCN # + dataADNI3
numfiles = len(dataFiles)
print('Found ', str(numfiles), ' nifti files')

Found  663  nifti files


In [3]:
# Match covariate information
import re
debug = False
cov_idx = [-1] * numfiles # list; array: np.full((numfiles, 1), -1, dtype=int)
print('Matching covariates for loaded files ...')
for i,id in enumerate(sid):
  p = [j for j,x in enumerate(dataFiles) if re.search('_%04d_' % id, x)] # translate ID numbers to four-digit numbers, get both index and filename
  if len(p)==0:
    if debug: print('Did not find %04d' % id) # did not find Excel sheet subject ID in loaded file selection
  else:
    if debug: print('Found %04d in %s: %s' % (id, p[0], dataFiles[p[0]]))
    cov_idx[p[0]] = i # store Excel index i for data file index p[0]
print('Checking for scans not found in Excel sheet: ', sum(x<0 for x in cov_idx))

labels = pd.DataFrame({'Group':grpbin}).iloc[cov_idx, :]
grps = pd.DataFrame({'Group':grp, 'RID':sid}).iloc[cov_idx, :]
amy_status = pd.DataFrame({'amy_pos':amy_bin}).iloc[cov_idx, :]

Matching covariates for loaded files ...
Checking for scans not found in Excel sheet:  0


In [4]:
# Load residualized data from disk
import h5py
import numpy as np
from pandas import DataFrame
from keras.utils import to_categorical
hf = h5py.File('orig_images_wb_mwp1_MNI.hdf5', 'r')
hf.keys # read keys
#labels = np.array(hf.get('labels')) # note: was of data frame type before
images = np.array(hf.get('images'))
hf.close()
print(images.shape)

Using TensorFlow backend.


(663, 100, 100, 120, 1)


In [5]:
labels = to_categorical(np.asarray(labels)) # use grps to access original labels

In [6]:
# filter scans by amyloid status
amy_filter = np.equal(np.transpose(amy_status.to_numpy()), labels[:,1])

# drop participants/entries by amy_filter
#dataFiles = [i for (i, v) in zip(dataFiles, np.squeeze(amy_filter)) if v] # filter list of files
#cov_idx = [i for (i, v) in zip(cov_idx, np.squeeze(amy_filter)) if v] # filter list of excel row indices
#labels = labels[np.squeeze(amy_filter), :]
#grps = grps.iloc[np.squeeze(amy_filter)]
#numfiles = len(dataFiles)
#print('Kept ', str(numfiles), ' nifti files')

In [7]:
# disable tensorflow deprecation warnings
import logging
logging.getLogger('tensorflow').disabled=True

In [8]:
# specify version of tensorflow
#%tensorflow_version 1.x
#%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
# downgrade to specific version
#!pip install tensorflow-gpu==1.15
#import tensorflow as tf
#print(tf.__version__)
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = False #True
session = tf.Session(config=config)
set_session(session)

1.15.0


In [9]:
# add new column of prediction value
grps['pred'] = np.nan #-1.0
# set predefined test indices from 20fold xval
test_indices = [np.array([ 10,  31,  47,  69,  86,  90, 115, 150, 164, 174, 191, 198, 222,
       228, 244, 258, 260, 264, 271, 288, 401, 415, 418, 439, 488, 526,
       576, 589, 594, 599, 610, 614, 629, 637]), np.array([ 14,  50,  53,  55,  76, 130, 145, 156, 173, 184, 189, 217, 247,
       265, 267, 268, 295, 310, 338, 358, 362, 424, 428, 433, 442, 477,
       505, 529, 533, 545, 561, 607, 625, 642]), np.array([  2,  19,  40,  87,  96, 103, 109, 125, 152, 153, 200, 219, 243,
       279, 284, 286, 291, 329, 374, 393, 399, 419, 427, 434, 484, 495,
       506, 557, 573, 593, 618, 630, 651, 660]), np.array([  5,  25,  30,  67, 120, 141, 161, 167, 176, 186, 207, 229, 257,
       261, 266, 296, 313, 325, 331, 346, 357, 425, 447, 454, 475, 476,
       546, 585, 586, 590, 604, 609, 615, 646]), np.array([ 21,  44,  59,  62,  91, 101, 121, 148, 158, 169, 193, 232, 259,
       262, 276, 277, 280, 282, 315, 378, 387, 464, 469, 493, 510, 513,
       535, 536, 548, 552, 581, 627, 632, 656]), np.array([  6,  26,  39,  49,  65,  97, 144, 162, 163, 182, 240, 250, 252,
       269, 327, 343, 345, 353, 365, 370, 398, 478, 482, 517, 518, 522,
       523, 524, 550, 553, 567, 587, 650, 653]), np.array([  4,  13,  17,  37,  71, 117, 138, 142, 149, 172, 202, 203, 204,
       208, 233, 253, 289, 306, 320, 380, 383, 414, 438, 465, 474, 491,
       494, 499, 520, 541, 563, 571, 579, 595]), np.array([ 41,  52,  56,  78, 105, 107, 128, 131, 133, 181, 201, 214, 224,
       238, 245, 290, 337, 372, 394, 396, 405, 421, 431, 437, 440, 444,
       451, 502, 521, 534, 582, 626, 644, 658]), np.array([  8,  24,  27,  51,  70,  77,  82, 166, 168, 185, 211, 231, 254,
       274, 322, 326, 344, 364, 382, 388, 392, 435, 450, 460, 485, 487,
       504, 528, 555, 559, 564, 631, 647, 654]), np.array([ 18,  58,  60,  75,  85, 108, 110, 118, 157, 206, 213, 235, 246,
       294, 302, 309, 311, 314, 385, 407, 409, 411, 455, 486, 509, 527,
       549, 568, 577, 592, 601, 613, 661]), np.array([ 16,  29,  88, 106, 122, 132, 135, 136, 160, 220, 251, 255, 273,
       275, 283, 318, 342, 355, 371, 390, 413, 420, 436, 441, 445, 470,
       507, 531, 551, 608, 638, 649, 657]), np.array([ 12,  36,  54,  61, 116, 126, 170, 171, 177, 194, 212, 241, 293,
       305, 312, 321, 361, 363, 369, 384, 453, 471, 473, 479, 537, 544,
       547, 556, 574, 591, 602, 605, 639]), np.array([ 35,  48,  63,  81,  98, 102, 111, 112, 124, 192, 205, 216, 218,
       223, 234, 237, 249, 285, 299, 367, 432, 448, 449, 456, 490, 503,
       532, 562, 565, 603, 622, 624, 635]), np.array([ 20,  57,  72,  74, 113, 119, 134, 155, 165, 196, 210, 215, 226,
       239, 297, 303, 307, 350, 360, 368, 412, 452, 458, 462, 514, 542,
       554, 570, 572, 584, 606, 640, 645]), np.array([  7,  34,  80,  89,  92, 104, 114, 129, 147, 209, 221, 256, 323,
       324, 330, 333, 359, 377, 379, 397, 416, 417, 443, 489, 492, 498,
       500, 519, 580, 636, 641, 652]), np.array([ 42,  43,  45,  64,  73,  94, 140, 154, 188, 281, 298, 334, 341,
       351, 375, 381, 391, 400, 404, 406, 422, 426, 468, 472, 481, 540,
       543, 575, 617, 628, 633, 643]), np.array([ 11,  66,  83, 123, 139, 151, 159, 178, 179, 242, 263, 272, 304,
       308, 352, 354, 376, 389, 403, 408, 459, 463, 467, 516, 578, 583,
       598, 616, 620, 621, 634, 659]), np.array([  0,   1,   3,  15,  23,  79,  95, 127, 183, 197, 225, 236, 319,
       328, 335, 336, 340, 348, 386, 395, 430, 446, 457, 466, 497, 538,
       566, 569, 596, 600, 612, 655]), np.array([  9,  22,  32,  33, 100, 137, 146, 180, 187, 190, 195, 230, 248,
       278, 287, 300, 301, 332, 347, 402, 410, 461, 483, 496, 508, 512,
       525, 539, 558, 560, 611, 648]), np.array([ 28,  38,  46,  68,  84,  93,  99, 143, 175, 199, 227, 270, 292,
       316, 317, 339, 349, 356, 366, 373, 423, 429, 480, 501, 511, 515,
       530, 588, 597, 619, 623, 662])]

In [10]:
# Split data into training/validation and holdout test data
from sklearn.model_selection import StratifiedKFold,train_test_split
import numpy as np
import gc
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt
#%matplotlib inline
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0" # model will be trained on GPU 0

import keras
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras import models
from keras.optimizers import Adam
from keras import initializers
from keras.callbacks import EarlyStopping
import tensorflow as tf
from statistics import mean,stdev
from sklearn.metrics import confusion_matrix

acc_AD, acc_MCI, auc_AD, auc_MCI = [], [], [], []
opt_acc_AD, opt_acc_MCI = [], []
#skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
batch_size = 8
k = 1
for test_idX in test_indices: #skf.split(X=images, y=grps.iloc[:, 0]): # split data as tenfold stratified cross-validation
    gc.collect() # run garbage collector
    test_idX = test_idX[np.squeeze(amy_filter)[test_idX]] # filter index vector by amy status
    testgrps = grps.iloc[test_idX, :]
    print(testgrps.RID) # prints test RID
    print('Distribution of diagnoses in holdout test data: [1=CN, 3=LMCI, 4=AD]')
    print(testgrps.Group.value_counts())
    
    testdat = images[test_idX, :]
    test_Y = labels[test_idX, :]

    print('validating model newmodel_rawdata_pretrained/newmodel_wb_cv%d.hdf5' % k)
    mymodel = models.load_model('newmodel_rawdata_pretrained/newmodel_wb_cv%d.hdf5' % k)
    k = k+1

    pred = mymodel.predict(testdat, batch_size=batch_size)
    grps.iloc[test_idX, 2] = pred[:, 1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    acc = dict()
    opt_acc = dict()
    thresh = dict()

    # redo AUC for binary comparison: AD vs. HC and MCI vs. HC
    for i in [3,4]:
      grpi = np.equal(testgrps.Group.to_numpy(dtype=np.int), np.ones((testgrps.shape[0],), dtype=np.int)*i)
      grp1 = np.equal(testgrps.Group.to_numpy(dtype=np.int), np.ones((testgrps.shape[0],), dtype=np.int))
      grpidx = np.logical_or(grpi, grp1)
      fpr[i], tpr[i], thresholds = roc_curve(test_Y[grpidx, 1], pred[grpidx, 1])
      thresh[i] = thresholds[np.argmax(tpr[i] - fpr[i])]
      print(thresh[i])
      roc_auc[i] = auc(fpr[i], tpr[i])
      acc[i] = np.mean((test_Y[grpidx, 1] == np.round(pred[grpidx, 1])).astype(int))*100
      opt_acc[i] = np.mean((test_Y[grpidx, 1] == (pred[grpidx, 1]>thresh[i]).astype(int)).astype(int))*100

    auc_AD.append(roc_auc[4])
    auc_MCI.append(roc_auc[3])
    acc_AD.append(acc[4])
    acc_MCI.append(acc[3])
    opt_acc_AD.append(opt_acc[4])
    opt_acc_MCI.append(opt_acc[3])
    
    print('AUC for MCI vs. CN = %0.3f' % roc_auc[3])
    print('AUC for AD vs. CN = %0.3f' % roc_auc[4])
    print('Acc for MCI vs. CN = %0.1f' % acc[3])
    print('Acc for AD vs. CN = %0.1f' % acc[4])
    print('confusion matrix (naive)')
    confmat = confusion_matrix(testgrps.Group-1, np.round(pred[:, 1]))
    print(confmat[:,(0,1)])
    print('oAcc for MCI vs. CN = %0.1f' % opt_acc[3])
    print('oAcc for AD vs. CN = %0.1f' % opt_acc[4])
    print('confusion matrix (optimized)')
    confmat = confusion_matrix(testgrps.Group-1, (pred[:, 1]>thresh[3]).astype(int)) ## use MCI threshold
    print(confmat[:,(0,1)])
    # break

1364    1030
1307     552
1315     667
1280     230
1473    4990
238     4201
528     4707
574     4801
621     4894
1320     698
1459    4899
1446    4542
1332     800
174     4079
640     4925
1310     610
1329     751
1327     741
4        413
209     4150
335     4384
362     4421
381     4449
407     4496
415     4506
458     4580
478     4609
Name: RID, dtype: int64
Distribution of diagnoses in holdout test data: [1=CN, 3=LMCI, 4=AD]
1    12
4     9
3     6
Name: Group, dtype: int64
validating model newmodel_rawdata_pretrained/newmodel_wb_cv1.hdf5
0.99545467
1.0
AUC for MCI vs. CN = 0.639
AUC for AD vs. CN = 0.981
Acc for MCI vs. CN = 66.7
Acc for AD vs. CN = 85.7
confusion matrix (naive)
[[9 3]
 [0 0]
 [3 3]
 [0 9]]
oAcc for MCI vs. CN = 77.8
oAcc for AD vs. CN = 57.1
confusion matrix (optimized)
[[12  0]
 [ 0  0]
 [ 4  2]
 [ 1  8]]
1607    5252
1395    1346
1501    5062
1295     361
406     4494
514     4672
543     4737
615     4887
658     4982
1392    1300
1400    1414
1397 

0.6472412
0.82377315
AUC for MCI vs. CN = 0.857
AUC for AD vs. CN = 1.000
Acc for MCI vs. CN = 75.0
Acc for AD vs. CN = 94.1
confusion matrix (naive)
[[8 1]
 [0 0]
 [3 4]
 [0 8]]
oAcc for MCI vs. CN = 75.0
oAcc for AD vs. CN = 94.1
confusion matrix (optimized)
[[9 0]
 [0 0]
 [4 3]
 [0 8]]
1492    5037
1333     830
1466    4959
1468    4968
239     4209
314     4353
602     4863
603     4867
633     4910
1289     296
199     4131
308     4346
351     4402
531     4713
542     4736
1428    4208
1316     668
237     4200
272     4276
294     4313
328     4376
364     4424
390     4466
Name: RID, dtype: int64
Distribution of diagnoses in holdout test data: [1=CN, 3=LMCI, 4=AD]
4    9
1    8
3    6
Name: Group, dtype: int64
validating model newmodel_rawdata_pretrained/newmodel_wb_cv12.hdf5
0.9999995
0.9999999
AUC for MCI vs. CN = 0.667
AUC for AD vs. CN = 0.681
Acc for MCI vs. CN = 57.1
Acc for AD vs. CN = 64.7
confusion matrix (naive)
[[4 4]
 [0 0]
 [2 4]
 [2 7]]
oAcc for MCI vs. CN = 71.4

In [11]:
# print model performance summary
print('Mean Acc for MCI vs. CN = %0.1f +/- %0.1f' % (mean(acc_MCI), stdev(acc_MCI)))
print('Mean Acc for AD vs. CN = %0.1f +/- %0.1f' % (mean(acc_AD), stdev(acc_AD)))
print('Mean oAcc for MCI vs. CN = %0.1f +/- %0.1f' % (mean(opt_acc_MCI), stdev(opt_acc_MCI)))
print('Mean oAcc for AD vs. CN = %0.1f +/- %0.1f' % (mean(opt_acc_AD), stdev(opt_acc_AD)))
print('Mean AUC for MCI vs. CN = %0.3f +/- %0.3f' % (mean(auc_MCI), stdev(auc_MCI)))
print('Mean AUC for AD vs. CN = %0.3f +/- %0.3f' % (mean(auc_AD), stdev(auc_AD)))

print('results details:')
results = pd.DataFrame({'Accuracy_MCI':acc_MCI, 'Accuracy_AD':acc_AD, 'AUC_MCI':auc_MCI, 'AUC_AD':auc_AD, 
                        'Opt_acc_MCI':opt_acc_MCI, 'Opt_acc_AD':opt_acc_AD})
print(results)

fpr_all, tpr_all, thresholds_all = roc_curve(labels[np.isfinite(grps.pred), 1], grps.pred[np.isfinite(grps.pred)])
thresh_all = thresholds_all[np.argmax(tpr_all - fpr_all)]
print(thresh_all)
roc_auc_all = auc(fpr_all, tpr_all)
acc_all = np.mean((labels[np.isfinite(grps.pred), 1] == np.round(grps.pred[np.isfinite(grps.pred)])).astype(int))*100
opt_acc_all = np.mean((labels[np.isfinite(grps.pred), 1] == (grps.pred[np.isfinite(grps.pred)]>thresh_all).astype(int)).astype(int))*100

confmat = confusion_matrix(grps.Group[np.isfinite(grps.pred)]-1, (grps.pred[np.isfinite(grps.pred)]>thresh_all).astype(int))
print('Naive accuracy: %0.1f %%' % acc_all)
print('confusion matrix (naive)')
print(confusion_matrix(grps.Group[np.isfinite(grps.pred)]-1, np.round(grps.pred[np.isfinite(grps.pred)]))[:,(0,1)])

print('Optimized accuracy: %0.1f %%' % opt_acc_all)
print('confusion matrix (optimized)')
print(confmat[:,(0,1)])

Mean Acc for MCI vs. CN = 70.6 +/- 8.6
Mean Acc for AD vs. CN = 78.3 +/- 11.6
Mean oAcc for MCI vs. CN = 77.3 +/- 9.9
Mean oAcc for AD vs. CN = 82.9 +/- 11.0
Mean AUC for MCI vs. CN = 0.811 +/- 0.128
Mean AUC for AD vs. CN = 0.903 +/- 0.111
results details:
    Accuracy_MCI  Accuracy_AD   AUC_MCI    AUC_AD  Opt_acc_MCI  Opt_acc_AD
0      66.666667    85.714286  0.638889  0.981481    77.777778   57.142857
1      81.250000    94.444444  0.857143  0.944444    81.250000   88.888889
2      66.666667    83.333333  0.802469  1.000000    72.222222   94.444444
3      68.750000    53.333333  0.857143  0.678571    81.250000   73.333333
4      66.666667    69.230769  0.629630  0.733333    50.000000   61.538462
5      63.157895    68.421053  0.806818  0.880682    84.210526   84.210526
6      78.571429    87.500000  0.875000  0.950000    71.428571   87.500000
7      89.473684    87.500000  0.977273  0.968750    89.473684   87.500000
8      77.777778    83.333333  0.909091  1.000000    77.777778   94